In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, TimestampType
from pyspark.sql.functions import col, to_timestamp, rand, lit, collect_list, array

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

In [3]:
# Define article custom schema
schemaArticle = StructType([
	StructField('_id', StringType(), True),
	StructField('title', StringType(), True),
	StructField('authors',
		ArrayType(
		StructType([
			 StructField('idAuth', StringType(), True),
			 StructField('org', StringType(), True)
		]), True)
	),
	StructField('n_citation', IntegerType(), True), 
	StructField('abstract', StringType(), True), 
	StructField('doi', StringType(), True),
	StructField('keywords', ArrayType(StringType()), True),
	StructField('isbn', StringType(), True),
	StructField('page_start', StringType(), True),
	StructField('page_end', StringType(), True),
	StructField('year', IntegerType(), True),
	StructField('fos', ArrayType(StringType()), True),
	StructField('references', ArrayType(StringType()), True),
	StructField('venue',
		StructType([
			 StructField('raw', StringType(), True),
			 StructField('type', IntegerType(), True),
			 StructField('issue', StringType(), True),
			 StructField('volume', StringType(), True),
			 StructField('publisher', StringType(), True)
		])
	),
])

In [4]:
#we decided to use import from schema to explicitly show data structure
df_articles = spark.read.schema(schemaArticle).json("./dblp_sample_filtered_spark.json", multiLine=True)

In [5]:
#issue, volume and publisher attributes inside venue are moved back in the root structure and removed from the inner struct
df_articles = df_articles.withColumn("issue", col("venue.issue")) \
						.withColumn("volume", col("venue.volume")) \
						.withColumn("publisher", col("venue.publisher")) \
						.withColumn("venue", col("venue").dropFields("issue", "volume", "publisher"))


In [6]:
#VENUES COLLECTION
#A new dataframe is created with attributes of venue and the _id of the article
#then it is all grouped by venue attributes and a list of the articles id for each venue is created
#finally we drop rows with null raw to delete inconsistent tuple
df_venues = df_articles.select("venue.raw", "venue.type", "_id") \
						.groupBy("raw", "type") \
						.agg(collect_list("_id").alias("artIds")) \
						.dropna(subset=["raw"])

In [7]:
# Create the schema for the DataFrame of Authors
schemaAuthors = StructType([
    StructField("_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("articles", ArrayType(StringType(), True), True),
    StructField("bio", StringType(), True),
    StructField("email", StringType(), True),
    StructField("orcid", StringType(), True),
    StructField("dob", TimestampType(), True)
])

In [8]:
#now we can keep only the raw attribute of the venue
df_articles = df_articles.withColumn("venue_raw", col("venue.raw")).drop("venue")

In [9]:
#we now add a generated field inside venues collection
#for each venue a random city is selected that should represent the place where the venue was held
citiesList = ["New York", "London", "Paris", "Berlin", "Madrid", "Rome", "Dublin", "Copenhagen", "Vienna", "Amsterdam", "Brussels", "Lisbon", "Prague", "Athens", "Budapest", "Warsaw", "Zurich", "Luxembourg", "Oslo", "Stockholm", "Helsinki", "Moscow", "Istanbul", "Kiev", "Minsk", "Belgrade", "Bucharest", "Sofia", "Tallinn", "Riga", "Vilnius", "Tbilisi", "Yerevan", "Baku", "Dubai", "Abu Dhabi", "Doha", "Manama", "Muscat", "Riyadh", "Jeddah", "Mecca", "Medina", "Kuala Lumpur", "Singapore", "Hong Kong", "Shanghai", "Beijing", "Tokyo", "Seoul", "Bangkok", "Manila"]
cities = array([lit(city) for city in citiesList])
df_venues = df_venues.withColumn("city", cities.getItem((rand() * len(citiesList)).cast("int")))

/usr/local/spark/python/pyspark/sql/column.py:419: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [10]:
#AUTHORS COLLECTION
#We simply import from json with autogenerated schema and the conversion from string to timestamp is applied
df_authors = spark.read.schema(schemaAuthors).json("./dblp_sample_reverted_filtered_spark.json", multiLine=True)
df_authors = df_authors.withColumn("dateofbirth", to_timestamp(df_authors["dob"], "yyyy-MM-dd'T'HH:mm:ss'Z'")) \
						.drop("dob") \
						.withColumnRenamed("dateofbirth", "dob")

In [11]:
df_articles.printSchema()
df_authors.printSchema()
df_venues.printSchema()
df_venues.show(10)

root
 |-- _id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- idAuth: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |-- n_citation: integer (nullable = true)
 |-- abstract: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: string (nullable = true)
 |-- page_start: string (nullable = true)
 |-- page_end: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- fos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- issue: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- venue_raw: string (nullable = true)

root
 |-- _id: string (nullable = true)
 |-- name: stri

# Data creation

In [12]:
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import from_unixtime, cast
from pyspark.sql.types import *

# Create a datetime object for the author's date of birth
dob = datetime.strptime("March 7, 1975", "%B %d, %Y")

# Convert the datetime object to a timestamp


# Create a new Row object with the values for the new author
new_author = Row(
    _id="638d9c6dae9ea0d19fad7a78",
    name="Emanuele Delle Valle ",
    nationality="it",
    # Set values for any other required columns
    articles=[],
    bio="Emanuele Della Valle holds a PhD in Computer Science from the \
        Vrije Universiteit Amsterdam and a Master degree in Computer Science\
        and Engineering from Politecnico di Milano. He is associate professor\
        at the Department of Electronics, Information and Bioengineering of\
        the Politecnico di Milano.",
    email="emanuele.dellavalle@gmail.com ",
    orcid="0000-0002-5176 -5885",
    dob=dob
)

# Add the new row to the DataFrame
df_authors = df_authors.union(spark.createDataFrame([new_author], schema = schemaAuthors))

In [13]:
df_authors.filter(col("_id") == "638d9c6dae9ea0d19fad7a78").show()

+--------------------+--------------------+-----------+--------+--------------------+--------------------+--------------------+-------------------+
|                 _id|                name|nationality|articles|                 bio|               email|               orcid|                dob|
+--------------------+--------------------+-----------+--------+--------------------+--------------------+--------------------+-------------------+
|638d9c6dae9ea0d19...|Emanuele Delle Va...|         it|      []|Emanuele Della Va...|emanuele.dellaval...|0000-0002-5176 -5885|1975-03-07 00:00:00|
+--------------------+--------------------+-----------+--------+--------------------+--------------------+--------------------+-------------------+



In [14]:
new_authors =  [Row("638db170ae9ea0d19fad7a79", "Politecnico di Milano"), Row("638db170ae9ea0d19fad7a7a", "Politecnico di Milano")]

new_article = Row(
    _id="638db237d794b76f45c77916",
    title="An extensive study of C-SMOTE, a Continuous Synthetic Minority Oversampling Technique for Evolving Data Streams",
    authors=new_authors,
    n_citation=3,
    abstract = "Streaming Machine Learning (SML) studies algorithms that update their models,\
        given an unbounded and often non-stationary flow of data performing a single pass. Online \
        class imbalance learning is a branch of SML that combines the challenges of both class imbalance\
        and concept drift. In this paper, we investigate the binary classification problem by rebalancing\
        an imbalanced stream of data in the presence of concept drift, accessing one sample at a time.",
    doi="10.1016/j.eswa.2022.116630",
    keywords=["Evolving Data Stream","Streaming","Concept drift","Balancing"],
    isbn="123-4-567-89012-3",
    page_start="39",
    page_end="46",
    year=2022,
    fos=["Computer Science","Stream Reasoning","Big Data"],
    references=["53e99fe4b7602d97028bf743","53e99fddb7602d97028bc085"],
    issue="1",
    volume="196",
    publisher="Elsevier",
    venue_raw="ESA"
)

# Add the new row to the DataFrame
df_articles = df_articles.union(spark.createDataFrame([new_article]))

In [16]:
df_articles.filter(col("_id") == "638db237d794b76f45c77916").show()

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-----------------+----------+--------+----+--------------------+--------------------+-----+------+---------+---------+
|                 _id|               title|             authors|n_citation|            abstract|                 doi|            keywords|             isbn|page_start|page_end|year|                 fos|          references|issue|volume|publisher|venue_raw|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-----------------+----------+--------+----+--------------------+--------------------+-----+------+---------+---------+
|638db237d794b76f4...|An extensive stud...|[{638db170ae9ea0d...|         3|Streaming Machine...|10.1016/j.eswa.20...|[Evolving Data St...|123-4-567-89012-3|        39|      46|2022|[Computer Science...|[53e99fe4b7602d97...|    1|

In [17]:
new_venue = Row(
    raw="ESA",
    type=1,
    artIds=["638db237d794b76f45c77916"],
    city="Montreal"
)

# Add the new row to the DataFrame
df_venues = df_venues.union(spark.createDataFrame([new_venue]))

In [18]:
df_venues.filter(col("raw") == "ESA").show()

+---+----+--------------------+---------+
|raw|type|              artIds|     city|
+---+----+--------------------+---------+
|ESA|   0|[53e99fddb7602d97...|Hong Kong|
|ESA|   1|[638db237d794b76f...| Montreal|
+---+----+--------------------+---------+



In [19]:
#adding the new article to the new author
import pyspark.sql.functions as f
df_authors = df_authors.withColumn(
    "articles",
    f.when(f.col("_id") == "638d9c6dae9ea0d19fad7a78",
        f.array_union(df_authors.articles, f.array(f.lit("638db237d794b76f45c77916"))))
    .otherwise(f.col("articles"))
)

In [22]:
df_authors.filter(col("_id") == "638d9c6dae9ea0d19fad7a78").show()

+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+
|                 _id|                name|nationality|            articles|                 bio|               email|               orcid|                dob|
+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+
|638d9c6dae9ea0d19...|Emanuele Delle Va...|         it|[638db237d794b76f...|Emanuele Della Va...|emanuele.dellaval...|0000-0002-5176 -5885|1975-03-07 00:00:00|
+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+



In [45]:
#increment number of citations
df_articles = df_articles.withColumn(
    "n_citation",
    f.when(f.col("_id") == "638db237d794b76f45c77916",
       df_articles.n_citation+1)
    .otherwise(f.col("n_citation"))
)

In [46]:
df_articles.filter(col("_id") == "638db237d794b76f45c77916").show()

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-----------------+----------+--------+----+--------------------+--------------------+-----+------+---------+---------+
|                 _id|               title|             authors|n_citation|            abstract|                 doi|            keywords|             isbn|page_start|page_end|year|                 fos|          references|issue|volume|publisher|venue_raw|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-----------------+----------+--------+----+--------------------+--------------------+-----+------+---------+---------+
|638db237d794b76f4...|An extensive stud...|[{638db170ae9ea0d...|         5|Streaming Machine...|10.1016/j.eswa.20...|[Evolving Data St...|123-4-567-89012-3|        39|      46|2022|[Computer Science...|[53e99fe4b7602d97...|    1|